Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
df2[:height] = convert(Vector{Float64}, df2[:height]);
df2[:weight] = convert(Vector{Float64}, df2[:weight]);
df2[:weight_s] = (df2[:weight] .- mean(df2[:weight])) / std(df2[:weight]);
df2[:weight_s2] = df2[:weight_s] .^ 2;

Define the Stan language model

In [5]:
weightsmodel = "
data{
    int N;
    real height[N];
    real weight_s2[N];
    real weight_s[N];
}
parameters{
    real a;
    real b1;
    real b2;
    real sigma;
}
model{
    vector[N] mu;
    sigma ~ uniform( 0 , 50 );
    b2 ~ normal( 0 , 10 );
    b1 ~ normal( 0 , 10 );
    a ~ normal( 178 , 100 );
    for ( i in 1:N ) {
        mu[i] = a + b1 * weight_s[i] + b2 * weight_s2[i];
    }
    height ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["a", "b1", "b2", "sigma"],
model=weightsmodel,  output_format=:mcmcchain);

Input data for cmdstan

In [7]:
heightsdata = Dict("N" => size(df2, 1), "height" => df2[:height],
"weight_s" => df2[:weight_s], "weight_s2" => df2[:weight_s2]);

Sample using cmdstan

In [8]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -36441.8, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -36437.9, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-co

Describe the draws

In [9]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
           Mean         SD       Naive SE       MCSE      ESS
    a 154.607996750 0.35981939 0.0056892441 0.0071717652 1000
   b1   5.836297555 0.27442159 0.0043389863 0.0041782214 1000
   b2  -0.010246876 0.22596945 0.0035728907 0.0043395335 1000
sigma   5.107366340 0.19829945 0.0031353896 0.0031505657 1000

Quantiles:
          2.5%        25.0%        50.0%        75.0%        97.5%   
    a 153.8889250 154.36900000 154.60400000 154.84625000 155.32805000
   b1   5.3007460   5.64918250   5.84176000   6.02085000   6.37030225
   b2  -0.4518894  -0.16156325  -0.00874418   0.13884025   0.43531998
sigma   4.7475043   4.96666750   5.10037000   5.23891000   5.51554550



End of `m4.5s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*